In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
"""
house_prices_full_corrected.py
Full single-file pipeline for Kaggle "House Prices - Advanced Regression Techniques".

- Kaggle-friendly paths
- Outlier handling
- Missing-value handling
- Feature engineering
- Target-safe target encoding
- Optuna tuning for LightGBM (50 trials)
- Bagged LightGBM, robust XGBoost, optional CatBoost, Ridge baseline
- Stacking meta-learner (Ridge)
- Writes submission.csv

Author: Generated for Santosh
Date: 2025-12-09
"""
# ---------------------------
# Imports & Settings
# ---------------------------
import os
import random
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
from scipy import stats
from scipy.special import boxcox1p

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

import xgboost as xgb
import lightgbm as lgb

# optional imports will be attempted later
warnings.filterwarnings("ignore")
SEED = 42
random.seed(SEED)
np.random.seed(SEED)


# ---------------------------
# Helpers
# ---------------------------
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))


def load_data_kaggle():
    train_path = "/kaggle/input/house-prices-advanced-regression-techniques/train.csv"
    test_path = "/kaggle/input/house-prices-advanced-regression-techniques/test.csv"
    if not Path(train_path).exists() or not Path(test_path).exists():
        raise FileNotFoundError("Dataset not found at Kaggle path. Adjust paths if running locally.")
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)
    return train_df, test_df


def target_encode_kfold(trn_ser, tst_ser, target, n_splits=5, min_samples_leaf=20, smoothing=10, noise_level=0.01):
    """
    K-fold target encoding (out-of-fold for train).
    Returns: (train_encoded_series, test_encoded_series)
    """
    oof = pd.Series(index=trn_ser.index, dtype=float)
    kf_local = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    for tr_idx, val_idx in kf_local.split(trn_ser):
        tr_vals = trn_ser.iloc[tr_idx]
        tr_target = target.iloc[tr_idx]
        averages = tr_target.groupby(tr_vals).agg(['mean', 'count'])
        smoothing_val = 1 / (1 + np.exp(-(averages['count'] - min_samples_leaf) / smoothing))
        prior = tr_target.mean()
        averages['smoothed'] = prior * (1 - smoothing_val) + averages['mean'] * smoothing_val
        mapping = averages['smoothed'].to_dict()
        oof.iloc[val_idx] = trn_ser.iloc[val_idx].map(mapping).fillna(prior)
    # Full-train mapping for test
    averages = target.groupby(trn_ser).agg(['mean', 'count'])
    smoothing_val = 1 / (1 + np.exp(-(averages['count'] - min_samples_leaf) / smoothing))
    prior = target.mean()
    averages['smoothed'] = prior * (1 - smoothing_val) + averages['mean'] * smoothing_val
    mapping = averages['smoothed'].to_dict()
    test_encoded = tst_ser.map(mapping).fillna(prior)
    # Add small noise to reduce overfitting
    oof = oof * (1 + noise_level * np.random.randn(len(oof)))
    test_encoded = test_encoded * (1 + noise_level * np.random.randn(len(test_encoded)))
    return oof, test_encoded


# ---------------------------
# Main pipeline
# ---------------------------
def main():
    print("Loading data...")
    train, test = load_data_kaggle()
    print(f"Train shape: {train.shape}, Test shape: {test.shape}")

    # Store IDs
    train_ID = train["Id"].copy()
    test_ID = test["Id"].copy()

    # Optional: remove obvious outliers (common trick)
    train = train.drop(train[(train["GrLivArea"] > 4000) & (train["SalePrice"] < 300000)].index, errors="ignore")

    # Drop Id columns
    train.drop(["Id"], axis=1, inplace=True)
    test.drop(["Id"], axis=1, inplace=True)

    # Target
    train["SalePrice"] = train["SalePrice"].astype(float)
    train["LogSalePrice"] = np.log1p(train["SalePrice"])
    y = train["LogSalePrice"].reset_index(drop=True)

    # Combine for consistent preprocessing
    ntrain = train.shape[0]
    all_data = pd.concat([train.drop(["SalePrice", "LogSalePrice"], axis=1), test], axis=0).reset_index(drop=True)
    print("Combined shape:", all_data.shape)

    # ---------------------------
    # Missing data heuristics
    # ---------------------------
    none_cols = [
        "Alley", "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2",
        "FireplaceQu", "GarageType", "GarageFinish", "GarageQual", "GarageCond",
        "PoolQC", "Fence", "MiscFeature", "MasVnrType"
    ]
    for col in none_cols:
        if col in all_data.columns:
            all_data[col] = all_data[col].fillna("None")

    if "GarageYrBlt" in all_data.columns:
        all_data["GarageYrBlt"] = all_data["GarageYrBlt"].fillna(0)

    # Numeric fill with median
    num_cols = all_data.select_dtypes(include=[np.number]).columns.tolist()
    for col in num_cols:
        if all_data[col].isnull().sum() > 0:
            all_data[col] = all_data[col].fillna(all_data[col].median())

    # Categorical fill with mode
    cat_cols = all_data.select_dtypes(include=["object"]).columns.tolist()
    for col in cat_cols:
        if all_data[col].isnull().sum() > 0:
            all_data[col] = all_data[col].fillna(all_data[col].mode()[0])

    # ---------------------------
    # Feature engineering
    # ---------------------------
    all_data["TotalSF"] = all_data.get("TotalBsmtSF", 0) + all_data.get("1stFlrSF", 0) + all_data.get("2ndFlrSF", 0)
    all_data["TotalPorchSF"] = (
        all_data.get("OpenPorchSF", 0)
        + all_data.get("EnclosedPorch", 0)
        + all_data.get("3SsnPorch", 0)
        + all_data.get("ScreenPorch", 0)
    )

    if "YrSold" in all_data.columns and "YearBuilt" in all_data.columns:
        all_data["YrSold"] = all_data["YrSold"].astype(int)
        all_data["YearBuilt"] = all_data["YearBuilt"].astype(int)
        all_data["AgeAtSale"] = all_data["YrSold"] - all_data["YearBuilt"]

    if "YearRemodAdd" in all_data.columns and "YrSold" in all_data.columns:
        all_data["YearsSinceRemod"] = all_data["YrSold"] - all_data["YearRemodAdd"]

    if "GarageCars" in all_data.columns and "GarageArea" in all_data.columns:
        all_data["GarageCars_per_Area"] = (all_data["GarageCars"] + 0.001) / (all_data["GarageArea"] + 0.001)

    if "OverallQual" in all_data.columns and "GrLivArea" in all_data.columns:
        all_data["Qual_by_Area"] = all_data["OverallQual"] * all_data["GrLivArea"]

    # ---------------------------
    # Target encoding for chosen features
    # ---------------------------
    te_candidates = [f for f in ["Neighborhood", "Exterior1st", "Exterior2nd", "SaleType", "MSSubClass"] if f in all_data.columns]
    for f in te_candidates:
        print("Target encoding:", f)
        train_ser = all_data.loc[: ntrain - 1, f].reset_index(drop=True)
        test_ser = all_data.loc[ntrain:, f].reset_index(drop=True)
        tr_enc, te_enc = target_encode_kfold(train_ser, test_ser, target=y, n_splits=5, min_samples_leaf=50, smoothing=20, noise_level=0.01)
        all_data.loc[: ntrain - 1, f + "_te"] = tr_enc.values
        all_data.loc[ntrain:, f + "_te"] = te_enc.values

    # ---------------------------
    # Skew transform numeric features
    # ---------------------------
    numeric_feats = all_data.select_dtypes(include=[np.number]).columns
    skewed_feats = all_data[numeric_feats].apply(lambda x: stats.skew(x.dropna())).sort_values(ascending=False)
    skewness = pd.DataFrame({"Skew": skewed_feats})
    skewed_features = skewness[abs(skewness["Skew"]) > 0.75].index.tolist()
    print(f"Found {len(skewed_features)} skewed numeric features to transform")
    for feat in skewed_features:
        try:
            all_data[feat] = boxcox1p(all_data[feat], 0.15)
        except Exception:
            pass

    # ---------------------------
    # Ordinal and small-cardinality encoding
    # ---------------------------
    ordinal_mappings = {
        "ExterQual": {"None": 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5},
        "ExterCond": {"None": 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5},
        "BsmtQual": {"None": 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5},
        "BsmtCond": {"None": 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5},
        "HeatingQC": {"None": 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5},
        "KitchenQual": {"None": 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5},
        "FireplaceQu": {"None": 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5},
        "GarageQual": {"None": 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5},
        "GarageCond": {"None": 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5},
        "PoolQC": {"None": 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5},
    }
    for col, mapping in ordinal_mappings.items():
        if col in all_data.columns:
            all_data[col] = all_data[col].map(mapping).astype(int)

    cat_cols = all_data.select_dtypes(include=["object"]).columns.tolist()
    for col in cat_cols:
        if all_data[col].nunique() <= 6:
            try:
                le = LabelEncoder()
                all_data[col] = le.fit_transform(all_data[col].astype(str))
            except Exception:
                pass

    # One-hot remaining categoricals
    all_data = pd.get_dummies(all_data)
    print("Shape after get_dummies:", all_data.shape)

    # ---------------------------
    # Split back
    # ---------------------------
    train_features = all_data[:ntrain].copy()
    test_features = all_data[ntrain:].copy()
    print("Train features:", train_features.shape, "Test features:", test_features.shape)

    # Align just in case
    train_features, test_features = train_features.align(test_features, join="left", axis=1, fill_value=0)

    # ---------------------------
    # CV setup
    # ---------------------------
    NFOLD = 5
    kf = KFold(n_splits=NFOLD, shuffle=True, random_state=SEED)

    n_train = train_features.shape[0]
    n_test = test_features.shape[0]

    # OOF and preds storage
    oof_lgb = np.zeros(n_train)
    pred_lgb = np.zeros(n_test)

    oof_xgb = np.zeros(n_train)
    pred_xgb = np.zeros(n_test)

    oof_ridge = np.zeros(n_train)
    pred_ridge = np.zeros(n_test)

    # Optional CatBoost
    use_catboost = False
    try:
        from catboost import CatBoostRegressor
        use_catboost = True
        oof_cat = np.zeros(n_train)
        pred_cat = np.zeros(n_test)
    except Exception:
        use_catboost = False

    # ---------------------------
    # LightGBM: Optuna tuning (quick study) + bagging
    # ---------------------------
    print("\nStarting LightGBM Optuna tuning and bagged training (50 trials Optuna)...")
    try:
        import optuna

        def lgb_objective(trial):
            params = {
                "objective": "regression",
                "metric": "rmse",
                "boosting": "gbdt",
                "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.05, log=True),
                "num_leaves": trial.suggest_int("num_leaves", 20, 200),
                "min_child_samples": trial.suggest_int("min_child_samples", 5, 200),
                "subsample": trial.suggest_float("subsample", 0.5, 1.0),
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
                "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
                "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
                "verbosity": -1,
                "seed": SEED,
            }

            kf_local = KFold(n_splits=3, shuffle=True, random_state=SEED)
            cv_scores = []
            for tr_idx, val_idx in kf_local.split(train_features):
                X_tr = train_features.iloc[tr_idx]
                X_val = train_features.iloc[val_idx]
                y_tr = y.iloc[tr_idx]
                y_val = y.iloc[val_idx]
                dtrain = lgb.Dataset(X_tr, label=y_tr)
                dval = lgb.Dataset(X_val, label=y_val)
                model = lgb.train(
                    params,
                    dtrain,
                    num_boost_round=2000,
                    valid_sets=[dval],
                    callbacks=[lgb.early_stopping(100), lgb.log_evaluation(period=0)],
                )
                best_it = model.best_iteration or 1000
                preds = model.predict(X_val, num_iteration=best_it)
                cv_scores.append(rmse(y_val, preds))
            return np.mean(cv_scores)

        study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=SEED))
        study.optimize(lgb_objective, n_trials=50, show_progress_bar=False)
        print("Optuna best params:", study.best_params)
        # translate to best params
        best_lgb_params = {
            "objective": "regression",
            "metric": "rmse",
            "boosting": "gbdt",
            "learning_rate": float(study.best_params["learning_rate"]),
            "num_leaves": int(study.best_params["num_leaves"]),
            "min_child_samples": int(study.best_params["min_child_samples"]),
            "subsample": float(study.best_params["subsample"]),
            "colsample_bytree": float(study.best_params["colsample_bytree"]),
            "reg_alpha": float(study.best_params["reg_alpha"]),
            "reg_lambda": float(study.best_params["reg_lambda"]),
            "verbosity": -1,
            "seed": SEED,
        }
    except Exception as e:
        print("Optuna not available or failed, falling back to default LGB params. Error:", e)
        best_lgb_params = {
            "objective": "regression",
            "metric": "rmse",
            "boosting": "gbdt",
            "learning_rate": 0.01,
            "num_leaves": 31,
            "min_child_samples": 20,
            "subsample": 0.8,
            "colsample_bytree": 0.8,
            "reg_alpha": 0.0,
            "reg_lambda": 0.0,
            "verbosity": -1,
            "seed": SEED,
        }

    # Bagging seeds
    lgb_seeds = [SEED, 7, 99]
    for seed_i in lgb_seeds:
        params = best_lgb_params.copy()
        params["seed"] = seed_i
        fold_oof = np.zeros(n_train)
        fold_pred = np.zeros(n_test)
        for fold, (tr_idx, val_idx) in enumerate(kf.split(train_features)):
            X_tr = train_features.iloc[tr_idx]
            X_val = train_features.iloc[val_idx]
            y_tr = y.iloc[tr_idx]
            y_val = y.iloc[val_idx]
            dtrain = lgb.Dataset(X_tr, label=y_tr)
            dval = lgb.Dataset(X_val, label=y_val)
            model = lgb.train(
                params,
                dtrain,
                num_boost_round=5000,
                valid_sets=[dval],
                callbacks=[lgb.early_stopping(200), lgb.log_evaluation(period=0)],
            )
            best_iter = model.best_iteration or 2000
            fold_oof[val_idx] = model.predict(X_val, num_iteration=best_iter)
            fold_pred += model.predict(test_features, num_iteration=best_iter) / NFOLD
            print(f"Seed {seed_i} | Fold {fold+1} LGB RMSE: {rmse(y_val, fold_oof[val_idx]):.5f}")
        # accumulate average across seeds
        oof_lgb += fold_oof / len(lgb_seeds)
        pred_lgb += fold_pred / len(lgb_seeds)

    print("LightGBM OOF RMSE:", rmse(y, oof_lgb))

    # ---------------------------
    # XGBoost (robust)
    # ---------------------------
    print("\nTraining XGBoost...")
    params_xgb = {
        "objective": "reg:squarederror",
        "learning_rate": 0.01,
        "max_depth": 6,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "seed": SEED,
        "verbosity": 0,
    }
    for fold, (tr_idx, val_idx) in enumerate(kf.split(train_features)):
        X_tr = train_features.iloc[tr_idx]
        X_val = train_features.iloc[val_idx]
        y_tr = y.iloc[tr_idx]
        y_val = y.iloc[val_idx]
        dtrain = xgb.DMatrix(X_tr, label=y_tr)
        dval = xgb.DMatrix(X_val, label=y_val)
        dtest = xgb.DMatrix(test_features)
        clf = xgb.train(
            params_xgb,
            dtrain,
            num_boost_round=10000,
            evals=[(dtrain, "train"), (dval, "eval")],
            early_stopping_rounds=200,
            verbose_eval=False,
        )
        # robust best iter
        best_iter = None
        if hasattr(clf, "best_iteration") and clf.best_iteration is not None:
            best_iter = int(clf.best_iteration)
        elif hasattr(clf, "best_ntree_limit") and clf.best_ntree_limit is not None:
            best_iter = int(clf.best_ntree_limit)
        else:
            best_iter = 10000
        try:
            oof_xgb[val_idx] = clf.predict(dval, iteration_range=(0, best_iter))
            pred_xgb += clf.predict(dtest, iteration_range=(0, best_iter)) / NFOLD
        except TypeError:
            oof_xgb[val_idx] = clf.predict(dval, ntree_limit=best_iter)
            pred_xgb += clf.predict(dtest, ntree_limit=best_iter) / NFOLD
        print(f"Fold {fold+1} XGB RMSE: {rmse(y_val, oof_xgb[val_idx]):.5f}")
    print("XGBoost OOF RMSE:", rmse(y, oof_xgb))

    # ---------------------------
    # Optional CatBoost (if installed)
    # ---------------------------
    if use_catboost:
        print("\nTraining CatBoost...")
        from catboost import CatBoostRegressor
        for fold, (tr_idx, val_idx) in enumerate(kf.split(train_features)):
            X_tr = train_features.iloc[tr_idx]
            X_val = train_features.iloc[val_idx]
            y_tr = y.iloc[tr_idx]
            y_val = y.iloc[val_idx]
            model = CatBoostRegressor(loss_function="RMSE", iterations=3000, learning_rate=0.03, depth=6, random_seed=SEED, early_stopping_rounds=200, verbose=0)
            model.fit(X_tr, y_tr, eval_set=(X_val, y_val))
            oof_cat[val_idx] = model.predict(X_val)
            pred_cat += model.predict(test_features) / NFOLD
            print(f"Fold {fold+1} CatBoost RMSE: {rmse(y_val, oof_cat[val_idx]):.5f}")
        print("CatBoost OOF RMSE:", rmse(y, oof_cat))

    # ---------------------------
    # Ridge baseline
    # ---------------------------
    print("\nTraining Ridge baseline...")
    for fold, (tr_idx, val_idx) in enumerate(kf.split(train_features)):
        X_tr = train_features.iloc[tr_idx]
        X_val = train_features.iloc[val_idx]
        y_tr = y.iloc[tr_idx]
        y_val = y.iloc[val_idx]
        clf = Ridge(alpha=20.0, random_state=SEED)
        clf.fit(X_tr, y_tr)
        oof_ridge[val_idx] = clf.predict(X_val)
        pred_ridge += clf.predict(test_features) / NFOLD
        print(f"Fold {fold+1} Ridge RMSE: {rmse(y_val, oof_ridge[val_idx]):.5f}")
    print("Ridge OOF RMSE:", rmse(y, oof_ridge))

    # ---------------------------
    # Stacking meta-learner
    # ---------------------------
    print("\nStacking meta-learner (Ridge)...")
    stack_oof_list = [oof_lgb, oof_xgb, oof_ridge]
    stack_pred_list = [pred_lgb, pred_xgb, pred_ridge]
    if use_catboost:
        stack_oof_list.append(oof_cat)
        stack_pred_list.append(pred_cat)

    X_oof = np.vstack(stack_oof_list).T
    X_test_meta = np.vstack(stack_pred_list).T

    meta_oof = np.zeros(n_train)
    meta_pred = np.zeros(n_test)

    for fold, (tr_idx, val_idx) in enumerate(kf.split(X_oof)):
        X_tr = X_oof[tr_idx]
        X_val = X_oof[val_idx]
        y_tr = y.iloc[tr_idx]
        y_val = y.iloc[val_idx]
        m = Ridge(alpha=5.0, random_state=SEED)
        m.fit(X_tr, y_tr)
        meta_oof[val_idx] = m.predict(X_val)
        meta_pred += m.predict(X_test_meta) / NFOLD
        print(f"Fold {fold+1} Meta RMSE: {rmse(y_val, meta_oof[val_idx]):.5f}")
    print("Meta OOF RMSE:", rmse(y, meta_oof))

    # ---------------------------
    # Final submission
    # ---------------------------
    final_preds_log = meta_pred
    final_preds = np.expm1(final_preds_log)
    submission = pd.DataFrame({"Id": test_ID, "SalePrice": final_preds})
    submission.to_csv("submission.csv", index=False)
    print("\nWrote submission.csv — ready to upload to Kaggle")
    print("Done.")


if __name__ == "__main__":
    main()


Loading data...
Train shape: (1460, 81), Test shape: (1459, 80)
Combined shape: (2917, 79)
Target encoding: Neighborhood
Target encoding: Exterior1st
Target encoding: Exterior2nd
Target encoding: SaleType
Target encoding: MSSubClass
Found 26 skewed numeric features to transform
Shape after get_dummies: (2917, 203)
Train features: (1458, 203) Test features: (1459, 203)


[I 2025-12-09 12:11:06,931] A new study created in memory with name: no-name-8851b36e-d979-4502-b5b9-07db67bf2168



Starting LightGBM Optuna tuning and bagged training (50 trials Optuna)...
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[2000]	valid_0's rmse: 0.142346
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[2000]	valid_0's rmse: 0.14751
Training until validation scores don't improve for 100 rounds


[I 2025-12-09 12:11:09,825] Trial 0 finished with value: 0.1408480243976647 and parameters: {'learning_rate': 0.01184431975182039, 'num_leaves': 192, 'min_child_samples': 148, 'subsample': 0.7993292420985183, 'colsample_bytree': 0.4936111842654619, 'reg_alpha': 2.5348407664333426e-07, 'reg_lambda': 3.3323645788192616e-08}. Best is trial 0 with value: 0.1408480243976647.


Did not meet early stopping. Best iteration is:
[2000]	valid_0's rmse: 0.132689
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1213]	valid_0's rmse: 0.14232
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1630]	valid_0's rmse: 0.145241
Training until validation scores don't improve for 100 rounds


...

Early stopping, best iteration is:
[575]	valid_0's rmse: 0.117653
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1081]	valid_0's rmse: 0.130003
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1358]	valid_0's rmse: 0.133234
Training until validation scores don't improve for 100 rounds


[I 2025-12-09 12:14:18,289] Trial 48 finished with value: 0.1269333782909717 and parameters: {'learning_rate': 0.013289543513718333, 'num_leaves': 43, 'min_child_samples': 34, 'subsample': 0.6984977041233017, 'colsample_bytree': 0.5992731499276404, 'reg_alpha': 1.4964852487826146e-05, 'reg_lambda': 0.0033588080354068277}. Best is trial 12 with value: 0.12182866238311672.


Early stopping, best iteration is:
[921]	valid_0's rmse: 0.117563
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[531]	valid_0's rmse: 0.128628
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[724]	valid_0's rmse: 0.129155
Training until validation scores don't improve for 100 rounds


[I 2025-12-09 12:14:21,547] Trial 49 finished with value: 0.12500559913006062 and parameters: {'learning_rate': 0.026373010902548025, 'num_leaves': 26, 'min_child_samples': 24, 'subsample': 0.6516047210107583, 'colsample_bytree': 0.5331071772915678, 'reg_alpha': 7.34875637057487e-08, 'reg_lambda': 0.025564299828799562}. Best is trial 12 with value: 0.12182866238311672.


Early stopping, best iteration is:
[326]	valid_0's rmse: 0.117235
Optuna best params: {'learning_rate': 0.02649072125632022, 'num_leaves': 23, 'min_child_samples': 7, 'subsample': 0.8836877669280698, 'colsample_bytree': 0.5686424561901412, 'reg_alpha': 1.638652911999463e-05, 'reg_lambda': 0.06642180887386759}
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[555]	valid_0's rmse: 0.127225
Seed 42 | Fold 1 LGB RMSE: 0.12722
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[715]	valid_0's rmse: 0.112152
Seed 42 | Fold 2 LGB RMSE: 0.11215
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[679]	valid_0's rmse: 0.123639
Seed 42 | Fold 3 LGB RMSE: 0.12364
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[791]	valid_0's rmse: 0.128592
Seed 42 | Fold 4 LGB RMSE: 0.12859
Training until validation scores don'